In [ ]:
from kafka import KafkaProducer, KafkaAdminClient
from kafka.admin import NewTopic
from kafka.errors import KafkaError
from kafka import KafkaConsumer
import random
import os, sys, datetime, time
import json

kafka_server_ip = "172.17.249.145"


# Kafka admin api

In [ ]:
class KafkaAdminAPIClient():
    '''
    admin client api implementation
    '''
    
    def __init__(self):
        
        # set up Kafka admin client
        self.admin_client = KafkaAdminClient(bootstrap_servers=[f'{kafka_server_ip}:9092'],
                                             client_id='admin'
                                            )
        
    def kafka_create_topic(self, topic_name, partitions, replication_factor):
        '''
        func to create topic
        using kafka admin client api
        '''
        
        topic_list = self.admin_client.list_topics()
        # Filter out internal topics that start with "__" ; -> example filters any consumer offsets
        if topic_name not in [t for t in topic_list if not t.startswith('__')]:
            print(f"Creating a new topic {topic_name}")
            # create new topic configuration
            new_topic = NewTopic(name = topic_name, 
                                 num_partitions = partitions, 
                                 replication_factor = replication_factor)

            # create new topic
            self.admin_client.create_topics(new_topics=[new_topic], validate_only=False)
            print(f"New kafka topic named {topic_name} created successfully....")
            
        else:
            
            print(f"{topic_name} already exists....")
            
    def kafka_delete_topic(self, topic_name):
        '''
        func to delete a topic using admin api
        '''    
            
        # Create the topic deletion request
        self.admin_client.delete_topics(topics=[topic_name])
        time.sleep(30)
                
        # Verify that the topic has been deleted
        if topic_name not in self.admin_client.list_topics():
            print(f"Topic {topic_name} has been successfully deleted.")
        else:
            print(f"Failed to delete topic {topic_name}.")
            
    def kafka_topics_list(self):
        '''
        func to get list of topics
        '''
        
        return self.admin_client.list_topics()
    
    def kafka_describe_topic(self, topic_name):
        
        return self.admin_client.describe_topics(topic_name)
    


In [ ]:
kafka_admin_client = KafkaAdminAPIClient()

In [ ]:
# create a kafka topic
kafka_admin_client.kafka_create_topic(topic_name = "topic1",
                                      partitions = 3,
                                      replication_factor = 1)

In [ ]:
# describe a kafka topic
kafka_admin_client.kafka_describe_topic(topic_name='topic1')

In [ ]:
# list kafka topics
kafka_admin_client.kafka_topics_list()

In [ ]:
# delete a kafka topic
kafka_admin_client.kafka_delete_topic(topic_name="topic2")

In [ ]:
# gracefully close kafka admin client
kafka_admin_client.admin_client.close()

# Kafka Producer example

In [ ]:
class KafkaProducerClient():
    
    def __init__(self):
        '''
        '''
        
        # kafka producer 
        self.producer = KafkaProducer(bootstrap_servers=f'{kafka_server_ip}:9092') #Same port as your Kafka server
        # kafka producer with serializer
        self.producer_serializer = KafkaProducer(bootstrap_servers=f'{kafka_server_ip}:9092',
                                                 value_serializer=lambda m: json.dumps(m).encode('utf-8'),
                                                 key_serializer=lambda m: m.encode('utf-8'),
                                                 security_protocol="PLAINTEXT") #Same port as your Kafka server
                
    
    def kafka_producer(self, topic_name, message_list):
        '''
        Func to publish data streams
        to kafka cluster
        '''
        
        stime = time.time()
        for line in message_list:
            # producer with callbacks
            self.producer.send( topic_name, 
                               str.encode(line)).add_callback(
                                   lambda x: print('Message delivered to partition {} with offset {}'.format(x.partition, x.offset))).add_errback(
                                       lambda excp: print (f"Failed to send message: {excp}"))
            self.producer.flush()
            
        print(f"Total time to publish {len(message_list)} messages : {time.time()-stime}")
        
        
    def kafka_producer_serializer(self, topic_name, json_list):
        '''
        Func to publish data streams
        to kafka cluster
        '''
        
        stime = time.time()
        for message in json_list:
            # producer with callbacks
            self.producer_serializer.send(topic_name, 
                               key = message['company'], value = message).add_callback(
                                   lambda x: print('Message delivered to partition {} with offset {}'.format(x.partition, x.offset))).add_errback(
                                       lambda excp: print (f"Failed to send message: {excp}"))
            self.producer.flush()
            
        print(f"Total time to publish {len(json_list)} messages : {time.time()-stime}")
        
        

In [ ]:
# create kafka producer
kafkaproducer_client = KafkaProducerClient()

In [ ]:
kafkaproducer_client.kafka_producer("topic1", message_list = ["hello world"])

In [ ]:
kafkaproducer_client.kafka_producer_serializer("topic2", message)

In [ ]:
# Create a message
message = [
    {
        "company": "3M",
        "description": "3M, based in Minnesota, may be best known for its Scotch tape and Post-It Notes, but it also produces sand paper, adhesives, medical products, computer screen filters, food safety items, stationery products and many products used in automotive, marine, and aircraft industries.",
        "initial_price": 44.28,
        "price_2002": 56.27,
        "price_2007": 95.85,
        "symbol": "MMM"
    },
    {
        "company": "Amazon.com",
        "description": "Amazon.com, Inc. is an online retailer in North America and internationally. The company serves consumers through its retail Web sites and focuses on selection, price, and convenience. It also offers programs that enable sellers to sell their products on its Web sites, and their own branded Web sites. In addition, the company serves developer customers through Amazon Web Services, which provides access to technology infrastructure that developers can use to enable virtually various type of business. Further, it manufactures and sells the Kindle e-reader. Founded in 1994 and headquartered in Seattle, Washington.",
        "initial_price": 89.38,
        "price_2002": 17.01,
        "price_2007": 93.43,
        "symbol": "AMZN"
    },
    {
        "company": "Campbell Soup",
        "description": "Campbell Soup is a worldwide food company, offering condensed and ready-to-serve soups; broth, stocks, and canned poultry; pasta sauces; Mexican sauces; canned pastas, gravies, and beans; juices and beverages; and tomato juices. Its customers include retail food chains, mass discounters, mass merchandisers, club stores, convenience stores, drug stores and other retail, and commercial and non-commercial establishments. Campbell Soup Company was founded in 1869 and is headquartered in Camden, New Jersey.",
        "initial_price": 37.0,
        "price_2002": 22.27,
        "price_2007": 36.4,
        "symbol": "CPB"
    },
    {
        "company": "Disney",
        "description": "The Walt Disney Company, founded in 1923, is a worldwide entertainment company, with movies, cable networks, radio networks, movie production, musical recordings and live stage plays. Disney also operates Walt Disney World in Florida and Disneyland in California, Disney Cruise Line, and international Disney resorts. Disney owns countless licenses and literary properties and publishes books and magazines.",
        "initial_price": 40.68,
        "price_2002": 15.24,
        "price_2007": 35.47,
        "symbol": "DIS"
    },
    {
        "company": "Dow Chemical",
        "description": "The Dow Chemical Company manufactures raw materials that go into consumer products and services. These materials include food and pharmaceutical ingredients, electronic displays, semiconductor packaging, water purification, insulation, adhesives, pest control, polyurethane, polystyrene (goes into plastics), and crude-oil based raw materials. Dow was founded in 1897 and is based in Midland, Michigan.",
        "initial_price": 38.83,
        "price_2002": 27.65,
        "price_2007": 44.67,
        "symbol": "DOW"
    },
    {
        "company": "Exxon Mobil",
        "description": "Exxon Mobil engages in the exploration and production of crude oil and natural gas, and manufacture of petroleum products. The company manufactures commodity petrochemicals. The company has operations in the United States, Canada/South America, Europe, Africa, Asia, and Australia/Oceania. Exxon Mobil Corporation was founded in1870 and is based in Irving, Texas.",
        "initial_price": 39.0,
        "price_2002": 32.82,
        "price_2007": 91.36,
        "symbol": "XOM"
    }
]